## I want to figure out why there is still a high energetic barrier for THR at lambda = 0 even after flattening torsions and 1,4s

### Let me first generate the htf with flattened torsions and 1,4s

In [1]:
import pickle

In [2]:
with open("/data/chodera/zhangi/perses_benchmark/neq/7/31/31_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [5]:
old_system = htf._topology_proposal.old_system
new_system = htf._topology_proposal.new_system

In [6]:
nb_force = old_system.getForce(3)
for i in range(nb_force.getNumExceptions()):
    p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(i)
    if p1 in [10, 11, 12, 13] or p2 in [10, 11, 12, 13]:
        if abs(p1-p2) == 3:
            nb_force.setExceptionParameters(i, p1, p2, 0, sigma, 0)

In [7]:
nb_force = new_system.getForce(3)
for i in range(nb_force.getNumExceptions()):
    p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(i)
    if p1 in [10, 13, 14, 15, 16, 17, 18, 19] or p2 in [10, 13, 14, 15, 16, 17, 18, 19]:
        if abs(p1-p2) == 3: 
            nb_force.setExceptionParameters(i, p1, p2, 0, sigma, 0)

In [9]:
# Build new htf
from perses.tests.test_topology_proposal import generate_atp
htf._topology_proposal._old_system = old_system
htf._topology_proposal._new_system = new_system

from perses.annihilation.relative import HybridTopologyFactory
htf = HybridTopologyFactory(topology_proposal=htf._topology_proposal,
                     current_positions=htf.old_positions(htf.hybrid_positions),
                     new_positions=htf.new_positions(htf.hybrid_positions),
                     use_dispersion_correction=False,
                     functions=None,
                     softcore_alpha=None,
                     bond_softening_constant=1.0,
                     angle_softening_constant=1.0,
                     soften_only_new=False,
                     neglected_new_angle_terms=[],
                     neglected_old_angle_terms=[],
                     softcore_LJ_v2=True,
                     softcore_electrostatics=True,
                     softcore_LJ_v2_alpha=0.85,
                     softcore_electrostatics_alpha=0.3,
                     softcore_sigma_Q=1.0,
                     interpolate_old_and_new_14s=False,
                     omitted_terms=None,
                     flatten_torsions=False)

INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:relative:Generating new system except

### First, let me try turning all torsion terms in the THR to see if a torsion scan suggests that the barrier is gone

In [ ]:
# See 08_debug_with_torsion_scan.ipynb

### Now, I will check that all non-valence terms for THR at lambda = 0 are zeroed

If charges and sigmas are not zeroed, there needs to be an exception

In [11]:
htf._atom_classes['unique_new_atoms']

{22, 23, 24, 25, 26, 27, 28, 29}

In [12]:
for i in range(htf.hybrid_system.getNumForces()):
    print(i, htf.hybrid_system.getForce(i))

0 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b1db351a570> >
1 <simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b1db351a570> >
2 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2b1db351a570> >
3 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b1db351a570> >
4 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2b1db351a570> >
5 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b1db351a570> >
6 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b1db351a570> >
7 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2b1db351a570> >
8 <simtk.ope

First let's check the NonbondedForce

In [14]:
nb_force = htf.hybrid_system.getForce(6)
for i in range(nb_force.getNumParticles()):
    print(i, nb_force.getParticleParameters(i))

0 [Quantity(value=0.1123, unit=elementary charge), Quantity(value=0.2649532787749369, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
1 [Quantity(value=-0.3662, unit=elementary charge), Quantity(value=0.3399669508423535, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
2 [Quantity(value=0.1123, unit=elementary charge), Quantity(value=0.2649532787749369, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
3 [Quantity(value=0.1123, unit=elementary charge), Quantity(value=0.2649532787749369, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
4 [Quantity(value=0.5972, unit=elementary charge), Quantity(value=0.3399669508423535, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
5 [Quantity(value=-0.5679, unit=elementary charge), Quantity(value=0.2959921901149463, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
6 [Quantity(value=-0.4157, unit=elementary charge), Quantity(value=0.3249998523775958, unit=nanometer), Quantity(value=0.0, unit

All the unique new atoms (22-29) have 0 charge and 0 epsilon 

Now let's check the CustomNonbondedForce

In [15]:
custom_nb_force = htf.hybrid_system.getForce(7)
for i in range(custom_nb_force.getNumParticles()):
    print(i, custom_nb_force.getParticleParameters(i))

0 (0.2649532787749369, 0.06568879999999999, 0.2649532787749369, 0.06568879999999999, 0.0, 0.0)
1 (0.3399669508423535, 0.4577296, 0.3399669508423535, 0.4577296, 0.0, 0.0)
2 (0.2649532787749369, 0.06568879999999999, 0.2649532787749369, 0.06568879999999999, 0.0, 0.0)
3 (0.2649532787749369, 0.06568879999999999, 0.2649532787749369, 0.06568879999999999, 0.0, 0.0)
4 (0.3399669508423535, 0.359824, 0.3399669508423535, 0.359824, 0.0, 0.0)
5 (0.2959921901149463, 0.87864, 0.2959921901149463, 0.87864, 0.0, 0.0)
6 (0.3249998523775958, 0.7112800000000001, 0.3249998523775958, 0.7112800000000001, 0.0, 0.0)
7 (0.10690784617684071, 0.06568879999999999, 0.10690784617684071, 0.06568879999999999, 0.0, 0.0)
8 (0.3399669508423535, 0.4577296, 0.3399669508423535, 0.4577296, 0.0, 0.0)
9 (0.2471353044121301, 0.06568879999999999, 0.2471353044121301, 0.06568879999999999, 0.0, 0.0)
10 (0.3399669508423535, 0.4577296, 0.3399669508423535, 0.0, 1.0, 0.0)
11 (0.2649532787749369, 0.06568879999999999, 0.2649532787749369, 0

Unique new atoms are not zeroed -- let's check to see if there are exceptions for these

In [ ]:
# custom_nb_force = htf.hybrid_system.getForce(7)
# for i in range(custom_nb_force.getNumParticles()):
#     print(i, custom_nb_force.getParticleParameters(i))

There are no exceptions in the CustomNonbondedForce...